# 2. Advanced Tutorials - 1. Customization - 3. Automatic differentiation and gradient tape

### 목차

1. Setup
2. Gradient tapes
 - 2-1 Recording control flow
 - 2-2 Higher-order gradients

URL : https://www.tensorflow.org/beta/tutorials/eager/automatic_differentiation

# 1. Setup

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

print(tf.__version__)

2.0.0-alpha0


# 2. Gradient tapes

In [2]:
x = tf.ones((2,2))

with tf.GradientTape() as t:
    t.watch(x)
    print(t)
    y = tf.reduce_sum(x) # 4
    print(y)
    z = tf.multiply(y,y) # z=y^2 = 16
    print(z)
    
dz_dx = t.gradient(z,x)
print(dz_dx)
for i in [0,1]:
    for j in [0,1]:
        assert dz_dx[i][j].numpy() == 8.0

tf.Tensor(4.0, shape=(), dtype=float32)
tf.Tensor(16.0, shape=(), dtype=float32)
tf.Tensor(
[[8. 8.]
 [8. 8.]], shape=(2, 2), dtype=float32)


In [3]:
x = tf.ones((2, 2))

with tf.GradientTape() as t:
    t.watch(x)
    y = tf.reduce_sum(x)
    z = tf.multiply(y, y)

dz_dy = t.gradient(z, y)
assert dz_dy.numpy() == 8.0
print(dz_dy)

tf.Tensor(8.0, shape=(), dtype=float32)


In [4]:
x = tf.constant(3.)
with tf.GradientTape(persistent=True) as t:
    t.watch(x)
    y = x*x
    z = y*y
dz_dx = t.gradient(z,x) # z = x^4 => 4x^3
print(dz_dx)
dz_dy = t.gradient(z,y)
print(dz_dy)
dy_dx = t.gradient(y,x)
print(dy_dx)
del t

tf.Tensor(108.0, shape=(), dtype=float32)
tf.Tensor(18.0, shape=(), dtype=float32)
tf.Tensor(6.0, shape=(), dtype=float32)


# 2-1 Recording control flow

In [5]:
def f(x,y):
    output = 1.
    for i in range(y):
        if i>1 and i<5:
            output = tf.multiply(output, x)
            print(i,':',output)
    return output

def grad(x,y):
    with tf.GradientTape() as t:
        t.watch(x)
        out = f(x,y)
    return t.gradient(out, x)

x = tf.convert_to_tensor(2.)
print(x)

assert grad(x,6).numpy() == 12. #x^3 => 3x^2
assert grad(x,5).numpy() == 12.
assert grad(x,4).numpy() == 4. # x^2 => 2x

tf.Tensor(2.0, shape=(), dtype=float32)
2 : tf.Tensor(2.0, shape=(), dtype=float32)
3 : tf.Tensor(4.0, shape=(), dtype=float32)
4 : tf.Tensor(8.0, shape=(), dtype=float32)
2 : tf.Tensor(2.0, shape=(), dtype=float32)
3 : tf.Tensor(4.0, shape=(), dtype=float32)
4 : tf.Tensor(8.0, shape=(), dtype=float32)
2 : tf.Tensor(2.0, shape=(), dtype=float32)
3 : tf.Tensor(4.0, shape=(), dtype=float32)


# 2-2 Higher-order gradients

In [6]:
x = tf.Variable(1.)

with tf.GradientTape() as t:
    with tf.GradientTape() as t2:
        y=x*x*x
        
    dy_dx = t2.gradient(y, x)
d2y_dx2 = t.gradient(dy_dx, x)

assert dy_dx.numpy() == 3.0
assert d2y_dx2.numpy() == 6.0